# Predicted Lithology - Transforming Data

2020.05.24

A first exercise of building an ML model out of Geochemistry to predict lithology

### Frame the problem
To predict lithology out of geochemistry assay data

### Load data using Pandas

In [18]:
import os
import pandas as pd

pd.options.mode.chained_assignment = None  # default='warn'
# source https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas 

SAMPLESPATH = os.path.join('datasets', 'uniw')

In [2]:
# function to load data

def loaddata(path, file):
    filetoretrieve = os.path.join(path, file)
    return pd.read_csv(filetoretrieve)

#### Samples

In [32]:
# load samples and tranform dataset
samplesall = loaddata(SAMPLESPATH, 'uniw_samples.csv')
# samplesall.head()

# select just the relevant fields
samples = samplesall[['SAMPLEID', 'HOLEID', 'SAMPFROM', 'SAMPTO', 'SAMPLETYPE', 'PRIORITY']]

# replace NULL/NaN Priorities with 0, reset indexes and make Priorities integer
samples['PRIORITY'] = samples['PRIORITY'].fillna(0)
samples.reset_index(drop=True)
samples['PRIORITY'] = samples['PRIORITY'].apply(lambda x : int(x))

/Users/rodrigo/Code/virtualenvironment/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (0,11) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [37]:
# find all Priorities used
samples['PRIORITY'].unique()

array([ 1,  0,  2, 10,  3,  5])

In [55]:
# transform HoleIDs
samples['HOLEID'] = samples['HOLEID'].apply(lambda x:x.replace('MONT', 'MARX').replace('WINU', 'XKCD')
                                            .replace('WIDI', 'SPIF').replace('RC18WIN', 'MUTL')
                                            .replace('RC17PAW', 'ROBO').replace('WB18WIN', 'GARF'))

In [56]:
samples.head()

,SAMPLEID,HOLEID,SAMPFROM,SAMPTO,SAMPLETYPE,PRIORITY
0,10415181,XKCD0010,386.0,387.0,DCore2QSw,1
1,10415182,XKCD0010,387.0,388.0,DCore2QSw,1
2,10415183,XKCD0010,388.0,389.0,DCore2QSw,1
3,10415184,XKCD0010,389.0,390.0,DCore2QSw,1
4,10415185,XKCD0010,390.0,391.0,DCore2QSw,1


In [57]:
# distinct SamplTypes
samples['SAMPLETYPE'].unique()

array(['DCore2QSw', 'DCore1QSw', 'Soil', 'DChip10Pct', 'DNotSampled',
       'DChipWhole', 'DCoreWhole', 'DChipUnk', 'DCore1QSp', 'DCore2QSp',
       'DChip12Pct', 'Water', 'DUnk', 'Composite', 'DChip8Pct', 'pXRF',
       'DCoreUnk', 'DChip1Q', 'DChip6Pct', 'DChipSpear'], dtype=object)

In [110]:
# list all HOLEIDs
samples['HOLEID'].unique()

array(['XKCD0010', 'XKCD0052', 'XKCD_007', 'XKCD0118', 'XKCD0061',
       'XKCD0012', 'XKCD0119', 'XKCD0141', 'XKCD0037', 'SPIF0009',
       'XKCD0075', 'XKCD0054', 'XKCD0313', 'SPIF0010', 'XKCD0193',
       'XKCD0194', 'XKCD0059', 'XKCD0159', 'XKCD0071', 'XKCD0348',
       'XKCD0353', 'XKCD0068', 'SPIF0011', 'XKCD0142', 'XKCD0073',
       'XKCD0079', 'XKCD0343', 'XKCD0342', 'XKCD0341', 'ROBO0001',
       'XKCD0324', 'XKCD0160', 'XKCD0015', 'ROBO0002', 'XKCD0051',
       'SPIF0012', 'XKCD0195', 'XKCD0064', 'XKCD0181', 'GARF0002',
       'SPIF0013', 'XKCD0043', 'XKCD0018', 'XKCD0182', 'XKCD0156',
       'XKCD0471', 'XKCD0157', 'XKCD0328', 'MUTL0001', 'XKCD0473',
       'XKCD0048', 'XKCD0158', 'XKCD0196', 'XKCD0294', 'XKCD0295',
       'XKCD0296', 'MUTL0002', 'XKCD0026', 'XKCD0297', 'XKCD0298',
       'XKCD0009', 'XKCD0299', 'XKCD0300', 'MUTL0003', 'XKCD0161',
       'SPIF0001', 'XKCD0053', 'XKCD0162', 'XKCD0067', 'SPIF0002',
       'XKCD0057', 'XKCD0164', 'SPIF0003', 'XKCD0293', 'XKCD00

In [89]:
samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177825 entries, 0 to 177824
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   SAMPLEID    177825 non-null  object 
 1   HOLEID      177825 non-null  object 
 2   SAMPFROM    177663 non-null  float64
 3   SAMPTO      177663 non-null  float64
 4   SAMPLETYPE  177825 non-null  object 
 5   PRIORITY    177825 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 8.1+ MB


In [123]:
# save transformed CSV
samples.to_csv(os.path.join('datasets', 'spif', 'spif_samples.csv'), sep=',', encoding='utf-8')

#### Lithology

In [111]:
# load Lithology
lithoall = loaddata(SAMPLESPATH, 'uniw_litho.csv')

# select just the relevant fields
lithoall = lithoall[['HOLEID', 'GEOLFROM', 'GEOLTO', 'PRIORITY', 'VALUE']]

# rename FROM, TO, VALUE as LITH
lithoall.columns = ['HOLEID', 'FROM', 'TO', 'PRIORITY', 'LITH']

# transform HOLEIDs
lithoall['HOLEID'] = lithoall['HOLEID'].apply(lambda x:x.replace('MONT', 'MARX').replace('WINU', 'XKCD')
                                            .replace('WIDI', 'SPIF').replace('RC18WIN', 'MUTL')
                                            .replace('RC17PAW', 'ROBO').replace('WB18WIN', 'GARF'))
lithoall.head()

,HOLEID,FROM,TO,PRIORITY,LITH
0,MARX0001,0.0,48.1,1,NotLogged
1,MARX0001,0.0,48.1,3,NotLogged
2,MARX0001,48.1,59.0,1,Sandstone
3,MARX0001,48.1,59.0,3,Sandstone
4,MARX0001,59.0,69.7,1,Claystone


In [115]:
lithoall.info()
# lithoall['PRIORITY'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15153 entries, 0 to 15152
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   HOLEID    15153 non-null  object 
 1   FROM      15153 non-null  float64
 2   TO        15153 non-null  float64
 3   PRIORITY  15153 non-null  int64  
 4   LITH      15153 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 592.0+ KB


In [78]:
# distinct lithologies
lithoall['LITH'].unique()

array(['NotLogged', 'Sandstone', 'Claystone', 'MetaSandstone',
       'MetaSiltstone', 'Quartzite', 'Psammite', 'Sand', 'Diamictite',
       'Tillite', 'LeachedCap', 'Clay', 'MassiveSulphide', 'Metapelite',
       'Conglomerate', 'Granite', 'Gravel', 'Mudstone', 'Arkose',
       'Gritstone', 'ClasticSediment', 'Breccia', 'Skarn', 'Silcrete',
       'QuartzVein', 'Dolerite', 'Ferricrete', 'MetaGritstone',
       'Saprolite', 'Arenite', 'Schist', 'Hornfels', 'FaultGouge',
       'Gneiss', 'Amphibolite', 'Gossan', 'Silt', 'Basalt',
       'MaficVolcanic', 'MaficIntrusive', 'MetaSediment', 'Dolomite',
       'Siltstone', 'AmphiboleSchist', 'Calcrete', 'Kaolinite',
       'IntermedIntrusive', 'BlackShale', 'CoreLoss', 'BrecciaTectonic',
       'BrecciaHydrothermal', 'BrecciaIntrusive', 'Vein'], dtype=object)

In [114]:
# make boolean dataframe where condition is true (i.e. Priority 1)
lithop1 = lithoall['PRIORITY'] == 1
lithop1

0         True
1        False
2         True
3        False
4         True
         ...  
15148     True
15149     True
15150     True
15151     True
15152     True
Name: PRIORITY, Length: 15153, dtype: bool

In [117]:
# new litho dataframe with Priorities 1 only, indexes reset
litho = lithoall[lithop1]
litho = litho.reset_index(drop=True)
litho

,HOLEID,FROM,TO,PRIORITY,LITH
0,MARX0001,0.0,48.1,1,NotLogged
1,MARX0001,48.1,59.0,1,Sandstone
2,MARX0001,59.0,69.7,1,Claystone
3,MARX0001,69.7,80.0,1,MetaSandstone
4,MARX0001,80.0,82.8,1,MetaSandstone
...,...,...,...,...,...
10402,XKCD0314,110.1,114.0,1,MetaSandstone
10403,XKCD0314,114.0,120.0,1,MetaSandstone
10404,XKCD0314,120.0,135.0,1,MetaSandstone
10405,XKCD0314,135.0,138.2,1,MetaSandstone


In [122]:
# save transformed CSV
litho.to_csv(os.path.join('datasets', 'spif', 'spif_litho.csv'), sep=',', encoding='utf-8')

#### Assays

In [90]:
# load Assays
assayall = loaddata(SAMPLESPATH, 'UNIW_assays.csv')

# select the relevant fields only
assayall = assayall[['SAMPLEID', 'NAME', 'PRIORITY', 'VALUE', 'DSC']]

assayall.head()

/Users/rodrigo/Code/virtualenvironment/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,SAMPLEID,NAME,PRIORITY,VALUE,DSC
0,10412498,Ag_4HSIMS_ppm,1,1.195,NaN
1,10412498,Ag_CNLAAS_ppm,1,0.840,NaN
2,10412498,Al_4HSIMS_pct,1,8.250,NaN
3,10412498,As_4HSIMS_ppm,1,4.070,NaN
4,10412498,As_PULCPX_ppm,1,50.000,<


In [103]:
# return only Priority 1
assay = assayall[assayall['PRIORITY'] == 1]

# disregard some assay types
assay = assay[~assay['NAME'].str.contains('ERROR')]
assay

,SAMPLEID,NAME,PRIORITY,VALUE,DSC
0,10412498,Ag_4HSIMS_ppm,1,1.195,NaN
1,10412498,Ag_CNLAAS_ppm,1,0.840,NaN
2,10412498,Al_4HSIMS_pct,1,8.250,NaN
3,10412498,As_4HSIMS_ppm,1,4.070,NaN
4,10412498,As_PULCPX_ppm,1,50.000,<
...,...,...,...,...,...
12708295,10813436,V_4HSIMS_ppm,1,73.000,NaN
12708296,10813436,W_4HSIMS_ppm,1,7.300,NaN
12708297,10813436,Y_4HSIMS_ppm,1,21.600,NaN
12708298,10813436,Zn_4HSIMS_ppm,1,53.000,NaN


In [121]:
assayrelevant.info()
# assay.info()
# assayall.info()

# <class 'pandas.core.frame.DataFrame'>
# RangeIndex: 12708300 entries, 0 to 12708299
# Data columns (total 8 columns):
#  #   Column         Dtype  
# ---  ------         -----  
#  0   SAMPLEID       object 
#  1   NAME           object 
#  2   PRIORITY       int64  
#  3   VALUE          float64
#  4   DSC            object 
#  5   LABJOBNO       object 
#  6   LOADDATE       object 
#  7   ANALYSISORDER  float64
# dtypes: float64(2), int64(1), object(5)
# memory usage: 775.7+ MB

# <class 'pandas.core.frame.DataFrame'>
# RangeIndex: 12708300 entries, 0 to 12708299
# Data columns (total 5 columns):
#  #   Column    Dtype  
# ---  ------    -----  
#  0   SAMPLEID  object 
#  1   NAME      object 
#  2   PRIORITY  int64  
#  3   VALUE     float64
#  4   DSC       object 
# dtypes: float64(1), int64(1), object(3)
# memory usage: 484.8+ MB

# <class 'pandas.core.frame.DataFrame'>
# Int64Index: 9796727 entries, 0 to 12708299
# Data columns (total 5 columns):
#  #   Column    Dtype  
# ---  ------    -----  
#  0   SAMPLEID  object 
#  1   NAME      object 
#  2   PRIORITY  int64  
#  3   VALUE     float64
#  4   DSC       object 
# dtypes: float64(1), int64(1), object(3)
# memory usage: 448.5+ MB

# <class 'pandas.core.frame.DataFrame'>
# Int64Index: 9505282 entries, 0 to 12708299
# Data columns (total 5 columns):
#  #   Column    Dtype  
# ---  ------    -----  
#  0   SAMPLEID  object 
#  1   NAME      object 
#  2   PRIORITY  int64  
#  3   VALUE     float64
#  4   DSC       object 
# dtypes: float64(1), int64(1), object(3)
# memory usage: 435.1+ MB

# <class 'pandas.core.frame.DataFrame'>
# RangeIndex: 7786855 entries, 0 to 7786854
# Data columns (total 6 columns):
#  #   Column    Dtype  
# ---  ------    -----  
#  0   index     int64  
#  1   SAMPLEID  object 
#  2   NAME      object 
#  3   PRIORITY  int64  
#  4   VALUE     float64
#  5   DSC       object 
# dtypes: float64(1), int64(2), object(3)
# memory usage: 356.5+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7786855 entries, 0 to 7786854
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   index     int64  
 1   SAMPLEID  object 
 2   NAME      object 
 3   PRIORITY  int64  
 4   VALUE     float64
 5   DSC       object 
dtypes: float64(1), int64(2), object(3)
memory usage: 356.5+ MB


In [109]:
# retrieve all assay types
fullassaylist = assay['NAME'].unique().tolist()
fullassaylist

['Ag_4HSIMS_ppm',
 'Ag_CNLAAS_ppm',
 'Al_4HSIMS_pct',
 'As_4HSIMS_ppm',
 'As_PULCPX_ppm',
 'Au_4HSIMS_ppm',
 'Au_CNLAAS_ppm',
 'Au_F30ICP_ppm',
 'Au_F50AAO_ppm',
 'Ba_4HSIMS_ppm',
 'Be_4HSIMS_ppm',
 'Bi_4HSIMS_ppm',
 'Ca_4HSIMS_pct',
 'Ca_PULCPX_pct',
 'Cd_4HSIMS_ppm',
 'Ce_4HSIMS_ppm',
 'Co_4HSIMS_ppm',
 'Cr_4HSIMS_ppm',
 'Cr_PULCPX_ppm',
 'Cs_4HSIMS_ppm',
 'Cu_4HOICP_pct',
 'Cu_4HSIMS_ppm',
 'Cu_PULCPX_ppm',
 'CuAS_SULAAS_pct',
 'CuCN_CNLAAS_pct',
 'CuRes_4HSAAS_pct',
 'CuT_SEQAAS_pct',
 'Fe_4HSIMS_pct',
 'Fe_PULCPX_pct',
 'Ga_4HSIMS_ppm',
 'Ge_4HSIMS_ppm',
 'Hf_4HSIMS_ppm',
 'In_4HSIMS_ppm',
 'K_4HSIMS_pct',
 'La_4HSIMS_ppm',
 'Li_4HSIMS_ppm',
 'Mg_4HSIMS_pct',
 'Mn_4HSIMS_ppm',
 'Mn_PULCPX_ppm',
 'Mo_4HSIMS_ppm',
 'Na_4HSIMS_pct',
 'Nb_4HSIMS_ppm',
 'Ni_4HSIMS_ppm',
 'Ni_PULCPX_ppm',
 'P_4HSIMS_pct',
 'Pb_4HSIMS_ppm',
 'Pb_PULCPX_ppm',
 'Pd_4HSIMS_ppm',
 'Pt_4HSIMS_ppm',
 'Rb_4HSIMS_ppm',
 'Re_4HSIMS_ppm',
 'S_4HSIMS_pct',
 'S_PULCPX_pct',
 'Sb_4HSIMS_ppm',
 'Sc_4HSIMS_ppm',
 'Se_4

In [118]:
# list methods to select them
distinctmethods = {i.split('_')[1] for i in fullassaylist}
distinctmethods

{'4HDIMS',
 '4HOICP',
 '4HSAAS',
 '4HSICP',
 '4HSIMS',
 'AQRIMS',
 'CILIMS',
 'CNLAAS',
 'F30AAS',
 'F30ICP',
 'F50AAO',
 'F50GRV',
 'F50ICP',
 'F50IMS',
 'HCLLEC',
 'KOFICH',
 'LMBICP',
 'LMBIMS',
 'LMBXRF',
 'LMDIMS',
 'LOIGRV',
 'LTBIMS',
 'NONGRV',
 'NONGSP',
 'NONLEC',
 'PULCPX',
 'PULFPX',
 'PULPYC',
 'Pass2mm',
 'Pass6mm',
 'Pass75um',
 'RAWFPX',
 'SEQAAS',
 'SPFICP',
 'SULAAS',
 'WAXGRV'}

In [120]:
# filter only for methods 4H% or F30% or F50%
assayrelevant = assay.loc[(assay['NAME'].str.contains('4H')) | (assay['NAME'].str.contains('F30')) | (assay['NAME'].str.contains('F50'))]
assayrelevant = assayrelevant.reset_index()
assayrelevant

,index,SAMPLEID,NAME,PRIORITY,VALUE,DSC
0,0,10412498,Ag_4HSIMS_ppm,1,1.195,NaN
1,2,10412498,Al_4HSIMS_pct,1,8.250,NaN
2,3,10412498,As_4HSIMS_ppm,1,4.070,NaN
3,5,10412498,Au_4HSIMS_ppm,1,0.971,NaN
4,7,10412498,Au_F30ICP_ppm,1,0.587,NaN
...,...,...,...,...,...,...
7786850,12708295,10813436,V_4HSIMS_ppm,1,73.000,NaN
7786851,12708296,10813436,W_4HSIMS_ppm,1,7.300,NaN
7786852,12708297,10813436,Y_4HSIMS_ppm,1,21.600,NaN
7786853,12708298,10813436,Zn_4HSIMS_ppm,1,53.000,NaN


In [124]:
# save transformed CSV
assayrelevant.to_csv(os.path.join('datasets', 'spif', 'spif_assay.csv'), sep=',', encoding='utf-8')

### Create one big flat table